# Bandits (Volume 2, Sections 17.2-3)
## Seong-Eun Cho

In [1]:
import numpy as np
import time
import scipy.stats as stats

In [2]:
if np.random.binomial(1, 0.1):
    print("success")
else:
    print("fail")
b = [4,3,2]
b[np.argmin(b)]

fail


2

In [8]:
class Bernoulli_bandits(object):
    """Models the Bernoulli bandit process/solution where one arm is pulled, the results are recorded,
    the next optimal arm (with the largest Gittins index) is then chosen and pulled, and the results
    recorded, and so on for T iterations"""
    
    def __init__(self, Theta, J, K, T, M, Beta):
        """Initialization method.
        
        Accepts:
            Theta: n-vector of success probabilities for each arm (0 indexed)
            J: n-vector of payouts for each arm (0 indexed)
            K: int, number of grid points in [0,J_i] to take each r from
            T: int, number of iterations to repeat the process
            M: int > T, at which to initialize the method compute_R()
            Beta: float, discount factor
            
        Initializes:
            Theta, J, wins, losses, K, T, M, Beta, total_winnings
            (optional): R (vector, if precompute compute_R() in final implementation)
        """
        self.n = len(Theta)
        self.Theta = Theta
        self.J = J
        self.K = K
        self.T = T
        self.M = M
        self.Beta = Beta
        self.wins = np.ones(self.n, dtype=int)
        self.losses = np.ones(self.n, dtype=int)
        self.total_winnings = 0
        self.R = None
        
    def initialize_R(self):
        """Optional method for precomputing compute_R() in final implentation. 
        PRO TIP: Don't use this method until rest of class is completed. After all, 
            'premature optimization is the root of all evil'."""
        R = {}
        for r in np.linspace(0,1,self.K):
            R[r] = self.compute_R(r)
        return R
    
    def pull(self, i):
        """Funtion that models a single pull of an n-armed bandit system.

        Accepts:
            i: index of arm that should be pulled. 

        Updates:
            total_winnings: value won from your pull
            wins[i]: number of successes of arm i
            losses[i]: number of failures of arm i
        """
        if np.random.binomial(1, self.Theta[i]):
            self.wins[i] += 1
            self.total_winnings += self.J[i]
        else:
            self.losses[i] += 1
        
    def compute_R(self, r):
        """Computes R value.

        Inputs:
            M: integer
            r: float

        Returns:
            R_values: array (M+1)x(M+1)
            """
        R = np.zeros((self.M+1,self.M+1))
        for a in range(self.M+1):
            b = self.M - a
            R[a,b] = 1/(1-self.Beta) * max([a/(a+b),r])
        for i in range(self.M-1, 0, -1):
            for a in range(i+1):
                b = i - a
                p1 = a/(a+b) * (1 + self.Beta*R[a+1,b]) + b/(a+b) * self.Beta*R[a,b+1]
                p2 = r/(1-self.Beta)
                R[a,b] = max([p1,p2])
        return R
        
    def gittins(self):
        """ Approximate gittins index to determine the next arm to pull.

        Parameters:
            J: lenth n array of floats (payouts)
            states: nx2 array of states
            M: integer
            K: integer, grid

        Returns:
            values: array of floats, corresponding to the Gittins index of each arm.
        """
        r_values = []
        for a,b in zip(self.wins, self.losses):
            diff = []
            for r in np.linspace(0,1,self.K):
                diff.append(abs(r/(1-self.Beta) - (a/(a+b) * (1 + self.Beta*self.R[r][a+1,b]) + b/(a+b) * self.Beta*self.R[r][a,b+1])))
            r_values.append(np.linspace(0,1,self.K)[np.argmin(diff)])
        return self.J * np.array(r_values)
        
    def simulate(self):
        """simulate multiple bandit pulls
        
        Returns:
            theta_hats: n-array of estimated probabilities
            self.total_winnings: (float) total amount won during play
        """
        self.R = self.initialize_R()
        for t in range(self.T):
            self.pull(np.argmax(self.gittins()))
        theta_hats = self.wins/(self.wins+self.losses)
        return theta_hats, self.total_winnings

In [6]:
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 100
T = 100
M = 105
Beta = 0.9
N = 20
Bandit = Bernoulli_bandits(Theta, J, K, T, M, Beta)
# Testing functionality of pull
'''
for i in range(N):
    Bandit.pull(0)
    Bandit.pull(1)
    Bandit.pull(2)
print(Bandit.wins)
print(Bandit.losses)
print(Bandit.total_winnings)
'''
# Testing functionality of compute_R
print(Bandit.compute_R(0.35))

[[ 0.          3.5         3.5        ...  3.5         3.5
   3.5       ]
 [10.          5.32762404  4.01744856 ...  3.5         3.5
   0.        ]
 [10.          6.71060486  5.13349643 ...  3.5         0.
   0.        ]
 ...
 [10.          9.90384615  9.80952381 ...  0.          0.
   0.        ]
 [10.          9.9047619   0.         ...  0.          0.
   0.        ]
 [10.          0.          0.         ...  0.          0.
   0.        ]]


In [9]:
# Do not modify the following test case, as it will be used in grading.
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 100
T = 100
M = 105
Beta = 0.9
N = 20
for i in range(N):
    print("trial ----------------{}-------------".format(i+1))
    gittins_bandit = Bernoulli_bandits(Theta, J, K, T, M, Beta)
    theta_hat, payout = gittins_bandit.simulate()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print()

trial ----------------1-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.42857143 0.64583333]
you won: $63

trial ----------------2-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.64285714 0.4       ]
you won: $63

trial ----------------3-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.5        0.33333333 0.68686869]
you won: $68

trial ----------------4-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.61333333 0.57142857]
you won: $60

trial ----------------5-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.55319149 0.33333333]
you won: $53

trial ----------------6-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.28571429 0.35294118 0.63076923]
you won: $52

trial ----------------7-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.48484848 0.56716418]
you won: $53


# Once the previous code is implemented for Bernoulli bandits, create a new class below which implements Thompson Sampling
## This should be relatively simple if you implemented Bernoulli bandits correctly. Don't forget to complete all parts of all the problems for full credit, particularly 17.10.

In [10]:
class Thompson_sampling(Bernoulli_bandits):
    def __init__(self, Theta, J, K, T, M, Beta):
        Bernoulli_bandits.__init__(self, Theta, J, K, T, M, Beta)
        
    def initialize_state(self, state_vector):
        self.wins = state_vector[:,0]
        self.losses = state_vector[:,1]
    # 17.8
    def thompson(self):
        samples = []
        for a,b in zip(self.wins, self.losses):
            samples.append(np.random.beta(a, b))
        return np.argmax(samples)
    # 17.9
    def simulate_thompson(self):
        for t in range(self.T):
            self.pull(self.thompson())
        theta_hats = self.wins/(self.wins+self.losses)
        return theta_hats, self.total_winnings
    

In [11]:
# Testing functionality of 17.9
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 100
T = 100
M = 105
Beta = 0.9
N = 20
for i in range(N):
    print("trial ----------------{}-------------".format(i+1))
    thompson_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    theta_hat, payout = thompson_bandit.simulate_thompson()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print()

trial ----------------1-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.41176471 0.65      ]
you won: $59

trial ----------------2-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.28571429 0.44444444 0.68888889]
you won: $65

trial ----------------3-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.23076923 0.47435897 0.53333333]
you won: $45

trial ----------------4-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.54545455 0.6        0.78666667]
you won: $74

trial ----------------5-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.2        0.42857143 0.75531915]
you won: $72

trial ----------------6-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.2        0.44444444 0.7826087 ]
you won: $74

trial ----------------7-------------
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.3125     0.5        0.62121212]
you won: $55


In [12]:
# 17.10 with K = 100, M = 105, T = 100
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 100
T = 100
M = 105
Beta = 0.9
N = 20
for i in range(N):
    print("trial ----------------{}-------------".format(i+1))
    print("Gittin simulation with K = {}, M = {}:".format(K, M))
    gittin_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t1 = time.time()
    theta_hat, payout = gittin_bandit.simulate()
    t2 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t2-t1))
    print()
    print("Thompson simulation:")
    thompson_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t3 = time.time()
    theta_hat, payout = thompson_bandit.simulate_thompson()
    t4 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t4-t3))
    print()

trial ----------------1-------------
Gittin simulation with K = 100, M = 105:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.41666667 0.78021978]
you won: $74
Simulation time: 1.2349801063537598seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.16666667 0.6031746  0.59459459]
you won: $58
Simulation time: 0.007215976715087891seconds

trial ----------------2-------------
Gittin simulation with K = 100, M = 105:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.25       0.52083333]
you won: $50
Simulation time: 1.2798528671264648seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.2        0.2        0.73958333]
you won: $70
Simulation time: 0.009247064590454102seconds

trial ----------------3-------------
Gittin simulation with K = 100, M = 105:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.33333333 0.77      ]
you won: $76
Simulation time: 1.

In [13]:
# 17.10 with K = 20, M = 22, T = 20
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 20
T = 20
M = 22
Beta = 0.9
N = 20
for i in range(N):
    print("trial ----------------{}-------------".format(i+1))
    print("Gittin simulation with K = {}, M = {}:".format(K, M))
    gittin_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t1 = time.time()
    theta_hat, payout = gittin_bandit.simulate()
    t2 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t2-t1))
    print()
    print("Thompson simulation:")
    thompson_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t3 = time.time()
    theta_hat, payout = thompson_bandit.simulate_thompson()
    t4 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t4-t3))
    print()

trial ----------------1-------------
Gittin simulation with K = 20, M = 22:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.5        0.33333333 0.73684211]
you won: $14
Simulation time: 0.018338918685913086seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.33333333 0.8       ]
you won: $15
Simulation time: 0.003566741943359375seconds

trial ----------------2-------------
Gittin simulation with K = 20, M = 22:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.66666667 0.5       ]
you won: $13
Simulation time: 0.028355121612548828seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.58333333 0.5       ]
you won: $10
Simulation time: 0.001157999038696289seconds

trial ----------------3-------------
Gittin simulation with K = 20, M = 22:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.2        0.64705882 0.25      ]
you won: $10
Simulation time: 0.02

In [14]:
# 17.10 with K = 50, M = 55, T = 10
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 50
T = 10
M = 55
Beta = 0.9
N = 20
for i in range(N):
    print("trial ----------------{}-------------".format(i+1))
    print("Gittin simulation with K = {}, M = {}:".format(K, M))
    gittin_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t1 = time.time()
    theta_hat, payout = gittin_bandit.simulate()
    t2 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t2-t1))
    print()
    print("Thompson simulation:")
    thompson_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t3 = time.time()
    theta_hat, payout = thompson_bandit.simulate_thompson()
    t4 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t4-t3))
    print()

trial ----------------1-------------
Gittin simulation with K = 50, M = 55:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.25 0.25 0.5 ]
you won: $3
Simulation time: 0.16497588157653809seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.25       0.88888889]
you won: $7
Simulation time: 0.0004868507385253906seconds

trial ----------------2-------------
Gittin simulation with K = 50, M = 55:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.25       0.66666667]
you won: $4
Simulation time: 0.14420700073242188seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.6        0.75      ]
you won: $7
Simulation time: 0.0004990100860595703seconds

trial ----------------3-------------
Gittin simulation with K = 50, M = 55:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.33333333 0.33333333 0.6       ]
you won: $5
Simulation time: 0.1595780849456787seconds



In [16]:
# 17.10 with K = 200, M = 210, T = 200
Theta = [0.2,0.5,0.7]
J = [1,1,1]
K = 200
T = 200
M = 210
Beta = 0.9
N = 20
for i in range(N):
    print("trial ----------------{}-------------".format(i+1))
    print("Gittin simulation with K = {}, M = {}:".format(K, M))
    gittin_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t1 = time.time()
    theta_hat, payout = gittin_bandit.simulate()
    t2 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t2-t1))
    print()
    print("Thompson simulation:")
    thompson_bandit = Thompson_sampling(Theta, J, K, T, M, Beta)
    t3 = time.time()
    theta_hat, payout = thompson_bandit.simulate_thompson()
    t4 = time.time()
    print("true probability: {}\nestimated probability: {}".format(Theta, theta_hat))
    print("you won: ${}".format(payout))
    print("Simulation time: {}seconds".format(t4-t3))
    print()

trial ----------------1-------------
Gittin simulation with K = 200, M = 210:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.4        0.33333333 0.65656566]
you won: $130
Simulation time: 8.737001895904541seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.14285714 0.578125   0.7037037 ]
you won: $130
Simulation time: 0.009381294250488281seconds

trial ----------------2-------------
Gittin simulation with K = 200, M = 210:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.25       0.25       0.69191919]
you won: $136
Simulation time: 8.767350196838379seconds

Thompson simulation:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.35      0.4       0.6746988]
you won: $124
Simulation time: 0.01355290412902832seconds

trial ----------------3-------------
Gittin simulation with K = 200, M = 210:
true probability: [0.2, 0.5, 0.7]
estimated probability: [0.25       0.475      0.69135802]
you won: $129
Simulation time: 9.2

# Feedback
10/10: Looks great!